# Register Candidate Model to SageMaker Model Registry

When you have a candidate model that is performing well according to your objective metric, you can register that version of your model to SageMaker Model Registry.   From there, it can be used for deployment for either batch or real-time use cases. 

A model version can be registered using the Studio Console, Boto3, or as a step in SageMaker Pipelines (which will be covered in a later chapter).

In this notebook, you'll perform the following tasks: 
 
 1. Create a Model Group which is group of versioned models
 2. Register a model version into the Model Group.  
 
For this, exercise we'll register the XGBoost model previously created in Chapter 5.  The same steps apply for the PyTorch model as well.   
 
First, you need to import the boto3 packages required. 

In [ ]:
import time
import os
from sagemaker import get_execution_role, session, image_uris
import boto3

region = boto3.Session().region_name

role = get_execution_role()

sm_client = boto3.client('sagemaker', region_name=region)

## 1. Create a Model Group

In [ ]:
import time
model_package_group_name = "air-quality-" + str(round(time.time()))
model_package_group_input_dict = {
 "ModelPackageGroupName" : model_package_group_name,
 "ModelPackageGroupDescription" : "model package group for air quality models",
 "Tags": [
     {
         "Key": "MLProject",
         "Value": "weather"
     }
 ]   
}
create_model_pacakge_group_response = sm_client.create_model_package_group(**model_package_group_input_dict)
print('ModelPackageGroup Arn : {}'.format(create_model_pacakge_group_response['ModelPackageGroupArn']))

## 2. Register a Model Version

First, we need to find the model_url that will be used as input to register the model version.  Typically this is included as part of a pipeline; however, in this case we are registering the model outside of a pipeline so we need to pull the data from our previous training job that resulted in a candidate model that is performing well according to our objective metric.

**Replace the variable below with the name of the training job from Chapter 5**

In [ ]:
#Example: 
#training_job = 'sagemaker-xgboost-2021-07-28-02-43-50-684'
training_job = 'REPLACE WITH NAME OF TRAINING JOB'

In [ ]:
training_job_response = sm_client.describe_training_job(
    TrainingJobName=training_job
)

In [ ]:
model_url=training_job_response['ModelArtifacts']['S3ModelArtifacts']
print('Model Data URL', model_url)

In [ ]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
xgboost_container = image_uris.retrieve(region=boto3.Session().region_name,
                          framework='xgboost', 
                          version='1.2-1')

print('XGBoost Container for Inference:', xgboost_container)

In [ ]:
modelpackage_inference_specification =  {
    "InferenceSpecification": {
      "Containers": [
         {
            "Image": xgboost_container,
             "ModelDataUrl": model_url
         }
      ],
      "SupportedContentTypes": [ "text/csv" ],
      "SupportedResponseMIMETypes": [ "text/csv" ],
   }
 }

create_model_package_input_dict = {
    "ModelPackageGroupName" : model_package_group_name,
    "ModelPackageDescription" : "Model to predict air quality ratings using XGBoost",
    "ModelApprovalStatus" : "PendingManualApproval"  
}
create_model_package_input_dict.update(modelpackage_inference_specification)

In [ ]:
create_model_package_response = sm_client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_mode_package_response["ModelPackageArn"]
print('ModelPackage Version ARN : {}'.format(model_package_arn))

Let's view the detailed of our registered model...

In [ ]:
sm_client.list_model_packages(ModelPackageGroupName=model_package_group_name)